In [1]:
import numpy as np
import os,random
from collections import Counter
import operator
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [2]:
path_list = os.listdir('./trainingDigits/')

In [3]:
'''读取文件'''
def img2vector(filename):
     returnVect = np.zeros((1,1024))
     fr = open(filename)
     for i in range(32):
         lineStr = fr.readline()
         for j in range(32):
             returnVect[0,32*i+j] = int(lineStr[j])
     return returnVect
img2vector('./trainingDigits/' + path_list[0]).shape

(1, 1024)

In [4]:
'''制作可用数据集'''
data_array = np.zeros(shape=(len(path_list), 1024))
label_array = np.zeros(shape=(len(path_list),))
for i in range(0, len(path_list)):
    data_array[i] = img2vector('./trainingDigits/' + path_list[i])
    label_array[i] = path_list[i][0]

index = [i for i in range(0, len(path_list))]
random.shuffle(index)
data_shuffle = data_array[index]
label_shuffle = label_array[index]
label_shuffle.shape
data_array.shape

(1934, 1024)

In [5]:
'''划分数据集'''
X_train = data_shuffle[0: 1500]
y_train = label_shuffle[0: 1500]
X_test = data_shuffle[1500: -1]
y_test = label_shuffle[1500: -1]
X_train.shape


(1500, 1024)

In [11]:
'''计算距离p=1曼哈顿，2为欧式距离'''
def L(x, y, p=2):
    # x1 = [1, 1], x2 = [5,1]
    if len(x) == len(y) and len(x) > 1:
        sum = 0
        for i in range(len(x)):
            sum += math.pow(abs(x[i] - y[i]), p)
        return math.pow(sum, 1 / p)
    else:
        return 0
'''面向对象建立模型'''
class KNN:
    def __init__(self, X_train, y_train, n_neighbors=4, p=2):

        self.n = n_neighbors
        self.p = p
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X):
        # 取出n个点
        knn_list = []
        for i in range(self.n):
            dist = np.linalg.norm(X - self.X_train[i], ord=self.p)
            knn_list.append((dist, self.y_train[i]))

        for i in range(self.n, len(self.X_train)):
            max_index = knn_list.index(max(knn_list, key=lambda x: x[0]))
            dist = np.linalg.norm(X - self.X_train[i], ord=self.p)
            if knn_list[max_index][0] > dist:
                knn_list[max_index] = (dist, self.y_train[i])

        # 统计
        knn = [k[-1] for k in knn_list]
        count_pairs = Counter(knn)
        max_count = sorted(count_pairs.items(), key=lambda x: x[1])[-1][0]
        return max_count

    def score(self, X_test, y_test):
        right_count = 0
        n = 10
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right_count += 1
        return right_count / len(X_test)

In [12]:
clf = KNN(X_train, y_train)

In [13]:
clf.score(X_test, y_test)

0.9745958429561201

In [9]:
'''简化了一下'''
def knn_model(x_test, x_data, y_data, k):
    # 计算样本数量
    x_size = x_data.shape[0]
    # 复制样本
    x_test = np.tile(x_test, (x_size, 1))
    # 计算欧式距离
    distance = np.square(x_test - x_data).sum(axis=1) ** 0.5
    # 排序索引
    disArg = distance.argsort()
    # 创建空字典
    classCount = {}
    # 循环k次将在范围内的值与对应的类别储存在字典中
    for i in range(k):
        # 将对应类别赋值作为字典的键
        voteClass = y_data[disArg[i]]
        # 给键赋值，如果没有给定的键，那么创建一个键并赋值给0
        classCount[voteClass] = classCount.get(voteClass, 0) + 1

    # 对字典进行降序排列
    dict_sort = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)

    # 取最小值
    knnclass = dict_sort[0][0]
    return knnclass

In [10]:
predictions = []
for i in range(X_test.shape[0]):
    predictions.append(knn_model(X_test[i], X_train, y_train, 4))

accuracy_score(y_test, predictions)


0.9815242494226328